###### PyCity Schools Analysis

---
* As a whole, schools with `higher budgets did not yield better test results`. By contrast, schools with higher spending  (\\$645-675 per student) actually underperformed compared to schools with smaller budgets (\\$<585 per student).


* As a whole, `smaller` (<1000 students) `and medium` sized (1001-2000 students) schools `dramatically out-performed large` sized  (2001-5000 students) `schools on passing math performances` (93.6% vs 70.0%).


* As a whole, `charter schools out-performed the public district schools across all metrics`. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school (on average 1,524 vs 3,853 students for district schools). 
---

In [31]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_data_to_load = "../Resources/schools_complete.csv"
student_data_to_load = "../Resources/students_complete.csv"

# Read School and Student Data Files and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the overall passing rate (overall average score), i.e. (average of the above two)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

`NOTES from the student`:
- The __overall passing rate__ was calculated based on the instructions - overall passing rate = (passing math score + passing reading score) / 2, which is different from the text included in the file PyCitySchools_starter.ipynb.
- For cleaner formatting, I chose only one decimal for percentages.


In [32]:
# Define a function to repeat the calculus of the mean in groups
def average(df, col, grp):
    return df[col].groupby(df[grp]).mean()

In [33]:
# Calculate the sum of all math score (70 or greater) between the total numbers of rows
pass_math_per = sum(school_data_complete.math_score >= 70) / school_data_complete.shape[0] * 100
# Calculate the sum of all reading score (70 or greater) between the total numbers of rows
pass_read_per = sum(school_data_complete.reading_score >= 70) / school_data_complete.shape[0] * 100

# Create a dataframe with the requested columns for the district summary
district_summary = pd.DataFrame(
    {"Total Schools" : [school_data_complete['school_name'].nunique()],
      # Apply format in the column Total Students to indicate thousands
     "Total Students" : ["{:,.0f}".format(school_data_complete['student_name'].count())],
      # Apply currency format in the column Total Budget 
     "Total Budget" : ["${:,.2f}".format(school_data_complete['budget'].unique().sum())],
     "Average Math Score" : ["{:,.1f}".format(school_data_complete['math_score'].mean())],
     "Average Reading Score" : ["{:,.1f}".format(school_data_complete['reading_score'].mean())],
     "% Passing Math" : ["{:,.1f}".format(pass_math_per)],
     "% Passing Reading" : ["{:,.1f}".format(pass_read_per)],
      # Calculate the overall average score as (% Passing Math + % Passing Reading) / 2
     "% Overall Passing Rate" : ["{:,.1f}".format((pass_math_per + pass_read_per) / 2)]
    })

# Show the results
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",79.0,81.9,75.0,85.8,80.4


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [34]:
# Get the school information: Name, School Type , Total Students(size) and Total School Budget
school_info = school_data.loc[:,["school_name", "type", "size", "budget"]]

# Calculate Per Student Budget as budget / size
school_info["Per Student Budget"] = school_info["budget"] / school_info["size"]

# Set the school_name as index
school_info.set_index("school_name", drop=True, inplace=True)

In [35]:
# Calculathe the average Math Score group by school name
school_math = average(school_data_complete, 'math_score', 'school_name')

# Calculathe the average Reading Score group by school name
school_read = average(school_data_complete, 'reading_score', 'school_name')

In [36]:
# Calculathe the Passing Math percentage
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['math_score'].count()
pass_math_per = pass_math / school_info['size'] * 100 

# Calculathe the Passing Reading percentage
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['reading_score'].count()
pass_read_per = pass_read / school_info['size'] * 100

In [37]:
# Concatenate the school information with the averages and passing percentages; calculate the overall passing rate
school_group = pd.concat([school_info, 
                          school_math, 
                          school_read, 
                          pass_math_per,
                          pass_read_per,
                          (pass_math_per + pass_read_per) / 2
                         ], axis=1, sort=True)

In [38]:
# Create a dataframe with the results and rename the columns for the school summary
school_summary = school_group.rename(columns={"type" : "School Type",
                                                    "size" : "Total Students",
                                                    "budget" : "Total School Budget",
                                                    "math_score" : "Average Math Score",
                                                    "reading_score" : "Average Reading Score",
                                                    0 : "% Passing Math",
                                                    1 :  "% Passing Reading",
                                                    2 : "% Overall Passing Rate"
                                                })

# Apply currency format in the columns Total School Budget and Per Student Budget
school_summary['Total School Budget'] = school_summary['Total School Budget'].map('${:,.2f}'.format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map('${:,.2f}'.format)

# Only one decimal in the percentages
pd.options.display.float_format = '{:,.1f}'.format

# Show the results
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0,81.0,66.7,81.9,74.3
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94.1,97.0,95.6
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66.0,80.7,73.4
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1,80.7,68.3,79.3,73.8
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93.4,97.1,95.3
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.3,80.9,66.8,80.9,73.8
Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,92.5,96.3,94.4
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,65.7,81.3,73.5
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.1,81.0,66.1,81.2,73.6
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6,95.9,95.3


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [39]:
# Order the dataframe by % Overall Passing Rate, show the best first
perform_school = school_summary.sort_values("% Overall Passing Rate", ascending=False)

# Show the first 5 observations (top five)
perform_school.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94.1,97.0,95.6
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4,83.8,93.3,97.3,95.3
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6,95.9,95.3
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93.4,97.1,95.3
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.3,84.0,93.9,96.5,95.2


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [40]:
# Order the dataframe by % Overall Passing Rate, show the worst first
perform_school = school_summary.sort_values("% Overall Passing Rate")

# Show the first 5 observations (top five)
perform_school.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8,80.7,66.4,80.2,73.3
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66.0,80.7,73.4
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,65.7,81.3,73.5
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.1,81.0,66.1,81.2,73.6
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1,80.7,68.3,79.3,73.8


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [41]:
# Get the school Name and set the index
school_name_only = school_data.loc[:,["school_name"]]
school_name_only.set_index("school_name", drop=True, inplace=True)

In [42]:
# Calculate the averages of each grade group by school_name
avg_math_9th = school_data_complete[school_data_complete['grade'] == '9th'].groupby('school_name')['math_score'].mean()
avg_math_10th = school_data_complete[school_data_complete['grade'] == '10th'].groupby('school_name')['math_score'].mean()
avg_math_11th = school_data_complete[school_data_complete['grade'] == '11th'].groupby('school_name')['math_score'].mean()
avg_math_12th = school_data_complete[school_data_complete['grade'] == '12th'].groupby('school_name')['math_score'].mean()

In [43]:
# Concatenate the series
math_grade = pd.concat([school_name_only, avg_math_9th, avg_math_10th, avg_math_11th, avg_math_12th], axis=1, sort=True)

# Set the names for the columns
math_grade.columns = ['9th', '10th', '11th', '12th']

# Only one decimal in the percentages
pd.options.display.float_format = '{:,.1f}'.format

# Show the results
math_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [44]:
# Calculate the averages of each grade group by school_name
avg_read_9th = school_data_complete[school_data_complete['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
avg_read_10th = school_data_complete[school_data_complete['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
avg_read_11th = school_data_complete[school_data_complete['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
avg_read_12th = school_data_complete[school_data_complete['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

In [45]:
# Concatenate the series
read_grade = pd.concat([school_name_only, avg_read_9th, avg_read_10th, avg_read_11th, avg_read_12th], axis=1, sort=True)

# Set the names for the columns
read_grade.columns = ['9th', '10th', '11th', '12th']

# Only one decimal in the percentages
pd.options.display.float_format = '{:,.1f}'.format

# Show the results
read_grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [46]:
# Set the bins and group names
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-614", "$615-644", "$645-675"]

# Create a dataframe with the requested columns (copy of the dataframe performing schools)
perform_school_nofmt = perform_school.loc[:,['Per Student Budget', 
                                             'School Type', 
                                             'Total Students', 
                                             'Average Math Score', 
                                             'Average Reading Score', 
                                             '% Passing Math', 
                                             '% Passing Reading', 
                                             '% Overall Passing Rate']]

# Remove the $ from the column to apply the function cut
perform_school_nofmt["Per Student Budget"] = perform_school_nofmt.iloc[:,0].str.replace('$', '').astype(float)

In [47]:
# Apply the function cut and insert a column named "Spending Ranges"
perform_school_nofmt["Spending Ranges"] = pd.cut(perform_school_nofmt["Per Student Budget"], spending_bins, labels=group_names)

In [48]:
# Calculate the averages group by Spending Ranges
score_math_avg = average(perform_school_nofmt,'Average Math Score','Spending Ranges')
score_read_avg = average(perform_school_nofmt,'Average Reading Score','Spending Ranges')
pass_math_avg = average(perform_school_nofmt,'% Passing Math','Spending Ranges')
pass_read_avg = average(perform_school_nofmt,'% Passing Reading','Spending Ranges')
pass_over_avg = average(perform_school_nofmt,'% Overall Passing Rate','Spending Ranges')

# Concatenate the averages 
school_spend_grp = pd.concat([score_math_avg, score_read_avg, pass_math_avg, pass_read_avg, pass_over_avg], axis=1, sort=True)

# Only one decimal in the percentages
pd.options.display.float_format = '{:,.1f}'.format

# Show the results
school_spend_grp

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges,,,,,
<$585,83.5,83.9,93.5,96.6,95.0
$585-614,83.6,83.9,94.2,95.9,95.1
$615-644,79.1,81.9,75.7,86.1,80.9
$645-675,77.0,81.0,66.2,81.1,73.6


## Scores by School Size

* Perform the same operations as above, based on school size.

In [49]:
# Set the bins and group names
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1001-2000)", "Large (2001-5000)"]

# Apply the function cut and insert a column named "School Size"
perform_school_nofmt["School Size"] = pd.cut(perform_school_nofmt["Total Students"], size_bins, labels=group_names)

In [50]:
# Calculate the averages group by School Size
score_math_avg = average(perform_school_nofmt,'Average Math Score','School Size')
score_read_avg = average(perform_school_nofmt,'Average Reading Score','School Size')
pass_math_avg = average(perform_school_nofmt,'% Passing Math','School Size')
pass_read_avg = average(perform_school_nofmt,'% Passing Reading','School Size')
pass_over_avg = average(perform_school_nofmt,'% Overall Passing Rate','School Size')

# Concatenate the averages 
school_size_grp = pd.concat([score_math_avg, score_read_avg, pass_math_avg, pass_read_avg, pass_over_avg], axis=1, sort=True)

# Only one decimal in the percentages
pd.options.display.float_format = '{:,.1f}'.format

# Show the results
school_size_grp

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.8,83.9,93.6,96.1,94.8
Medium (1001-2000),83.4,83.9,93.6,96.8,95.2
Large (2001-5000),77.7,81.3,70.0,82.8,76.4


## Scores by School Type

* Perform the same operations as above, based on school type.

In [51]:
# Calculate the averages group by School Type
score_math_avg = average(perform_school_nofmt,'Average Math Score','School Type')
score_read_avg = average(perform_school_nofmt,'Average Reading Score','School Type')
pass_math_avg = average(perform_school_nofmt,'% Passing Math','School Type')
pass_read_avg = average(perform_school_nofmt,'% Passing Reading','School Type')
pass_over_avg = average(perform_school_nofmt,'% Overall Passing Rate','School Type')

# Concatenate the averages 
school_type_grp = pd.concat([score_math_avg, score_read_avg, pass_math_avg, pass_read_avg, pass_over_avg], axis=1, sort=True)

# Only one decimal in the percentages
pd.options.display.float_format = '{:,.1f}'.format

# Show the results
school_type_grp

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.5,83.9,93.6,96.6,95.1
District,77.0,81.0,66.5,80.8,73.7
